<a href="https://colab.research.google.com/github/MeeTK26/30DaysOfMLDL/blob/main/AudioSpeachActorOfLibrosa(day29).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np

In [17]:
RAVDESS_DIR='/content/drive/MyDrive/Audio_Speech_Actors_01-24'
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

In [20]:
import librosa
import librosa.display

def extract_features(file_path, max_pad_len=174):
    audio, sample_rate = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant') if pad_width > 0 else mfccs[:, :max_pad_len]
    return mfccs

In [21]:
import os
import resampy
X, y = [], []
for root, _, files in os.walk(RAVDESS_DIR):
    for file in files:
        if file.endswith(".wav"):
            emotion = emotions[file.split("-")[2]]
            feature = extract_features(os.path.join(root, file))
            X.append(feature)
            y.append(emotion)

In [22]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

X = np.array(X)
y = np.array(y)
le = LabelEncoder()
y = le.fit_transform(y)
y = tf.keras.utils.to_categorical(y, num_classes=len(emotions))

In [23]:
from sklearn.model_selection import train_test_split

X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape,Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout, TimeDistributed

model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], 1)),
    MaxPooling2D((2, 2)),
    Reshape((-1, 64)),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(len(emotions), activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 125ms/step - accuracy: 0.1442 - loss: 2.1084 - val_accuracy: 0.1667 - val_loss: 2.0498
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 119ms/step - accuracy: 0.1509 - loss: 2.0617 - val_accuracy: 0.1944 - val_loss: 2.0398
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step - accuracy: 0.1533 - loss: 2.0664 - val_accuracy: 0.1979 - val_loss: 2.0282
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 102ms/step - accuracy: 0.1662 - loss: 2.0527 - val_accuracy: 0.1667 - val_loss: 2.0264
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.1795 - loss: 2.0129 - val_accuracy: 0.1944 - val_loss: 1.9890
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 102ms/step - accuracy: 0.2029 - loss: 2.0053 - val_accuracy: 0.1875 - val_loss: 1.9736
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 112ms/step - accuracy: 0.2079 - loss: 1.9908 - val_accuracy: 0.2361 - val_loss: 1.9737
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.1912 - loss: 2.0016 - val_accuracy: 0